In [1]:
# Formula is simple Eij = Dij * (1 + Aij + Bij + Cij) where Aij is the ith and jth entry from the A matrix which we learn.


In [8]:
#So what we need to do is read in each chromosome.
#initialize all of our starting values.
#iterate over the entire chromosome and update distance
#iterate over the entire chromosome and update bias
#iterate again and update C-score
#repeat until error tolerance is met.

In [29]:
import matplotlib
%matplotlib inline
#matplotlib.rcParams['figure.figsize'] = (10,10)

In [30]:
import numpy as np

celltype = "HCT116-untreated"
#hic_directory = "/Zulu/mike/dumped-hic/GM12878/"
hic_directory = "/Zulu/mike/dumped-hic/HCT116/"
#hic_prefix = "GM12878-untreated-NOKR-dumped"
#hic_prefix = "GM12878-untreated-q30-KR-dumped"
hic_prefix = "HCT116-untreated-q30-KR-dumped"

male = True

sizefile = "/Zulu/mike/hg19.chrom.sizes"

resolution = 100000
small_resolution = 100000

distance_min = 0#5000000
distance_min_bins = int(distance_min/resolution)
distance_max = 1000000000
distance_max_bins = int(distance_max/resolution)
#print(distance_min_bins)
distance_expand_rate = 0.04

chrstart = 0
chrstop = 23

tolerance = 10000
max_iter = 10

ignore_threshold = 0.01

#100kb
if resolution == 100000:
    correction = "sums nonzero medians quant"
    medianhighstd = 3
    medianlowstd= 3
    highstd = 3.5
    lowstd = 2.5
    nz_low_threshold = 3
    nz_high_threshold = 3
    zscore_threshold = 10

#25kb
elif resolution == 25000:
    correction = "sums nonzero"
    medianhighstd = 3
    medianlowstd= 3
    highstd = 5
    lowstd = 5
    nz_low_threshold = 3
    nz_high_threshold = 3
    zscore_threshold = 25
else:
    print("woah nelly bad resolution")

ignorebins = [False]


#number of potential compartments
#signal files
filenames = []
#signal names
names = []
filenames.append("hct116-H3K9me3-ENCFF402WZH.bigWig")
names.append("H3K9me3")
filenames.append("hct116-H3K27me3-ENCFF030SYQ.bigWig")
names.append("H3K27me3")
filenames.append("hct116-H3K27ac-ENCFF225QAB.bigWig")
names.append("H3K27ac")
#filenames.append("GSE86165_HCT116_gro.bw")
#names.append("Gro")
#filenames.append("hct116-CTCF-ENCFF695DDS.bigWig")
#names.append("CTCF")
#filenames.append("hct116-H2AFZ-ENCFF996RJO.bigWig")
#names.append("H2AFZ")
#filenames.append("hct116-H3K9me2-ENCFF854QMM.bigWig")
#names.append("H3K9me2")
#filenames.append("hct116-H3K4me2-ENCFF563OKQ.bigWig")
#names.append("H3K4me2")
#filenames.append("hct116-H3K79me2-ENCFF127XQD.bigWig")
#names.append("H3K79me2")
#filenames.append("hct116-H4K20me1-ENCFF355TAA.bigWig")
#names.append("H4K20me1")
#filenames.append("hct166-H3K4me1-ENCFF285DIL.bigWig")
#names.append("H3K4me1")
#filenames.append("hct116-H3K4me3-ENCFF057GFU.bigwig")
#names.append("H3K4me3-1")
#filenames.append("hct116-H3K4me3-ENCFF859AQD.bigwig")
#names.append("H3K4me3-2")
#filenames.append("hct116-H3K36me3-ENCFF062CBC.bigWig")
#names.append("H3K36me3")

comp_num = len(filenames)

#smooth = 21

fft_fc = 0.01
fft_b = 0.08
fft_N = int(np.ceil((4 / fft_b)))
if not fft_N % 2: fft_N += 1
fft_n = np.arange(fft_N)

ignoreplots = False

tiles = 21

clims = [0,1]
slims = [0,10]
blims = [0,2]


In [31]:
sizes = open(sizefile,'r')

chrnames = []
chrsizes = []
for line in sizes:
    li = line.split()
    if li[0] == 'Y' or li[0] == 'MT':
        continue
    chrnames.append(li[0])
    chrsizes.append(int(li[1]))

sizes.close()

print(chrnames)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X']


In [32]:
import math
#determine size of each chromosome in bins:
chrsizebins = []

for i in range(len(chrnames)):
    s = int(math.ceil(chrsizes[i]/resolution + 1))
    chrsizebins.append(s)
    
totalsize = sum(chrsizebins)
print(totalsize)
print(chrsizebins)

30399
[2494, 2433, 1982, 1913, 1811, 1713, 1593, 1465, 1414, 1357, 1352, 1340, 1153, 1075, 1027, 905, 813, 782, 593, 632, 483, 515, 1554]


In [33]:
import numpy as np
#inits
max_size = max(chrsizebins)
print(max_size)

distance_indices = []#[0]*distance_min_bins
distance_ranges = []
#d = int(distance_min_bins)
d = 0
count = 0
while d < max_size:
    d1 = d
    distance_ranges.append([d,d1])
    distance_indices.extend([count]*(d1-d+1))
    #print(len(distance_indices))
    d = d1+1
    #print(d1)
    count += 1
    

#distance_range_count = len(distance_ranges)
#print(distance_range_count)
#print(distance_ranges)
#print(len(distance_indices))
#print(distance_indices[210:230])
#D = list(np.random.rand(max_size) * 2)
D = list(np.zeros(max_size))
#S = list(np.random.rand(distance_range_count,comp_num))
#S = list(np.random.rand(distance_range_count,comp_num) * 2)
#S = list(np.ones((distance_range_count,comp_num)) * 2)
#this is for each distance but you need to use chrsize, distance_min, and merge rate to calculate the actual size
#init should follow a power law decay? slightly different because distances are expanding
#B = [1]*max_size
#B = list(np.random.rand(max_size) * 2)
#this is bias for each bin
#should start as 1
#C = [0.01]*max_size
#C = list(np.random.rand(max_size,comp_num))
#C = []
#this is C-score,
#should start as 0, same size as B

M = []
for i in range(comp_num):
    M.append(np.zeros((tiles,tiles)))
    
#print(M)

2494


In [34]:
import pandas as pd
import pyBigWig
import math
from os.path import commonprefix

#small_resolution = 5000
resolution_ratio = resolution//small_resolution


print(filenames)
#filenames = ["hct116-H3K9me3-ENCFF402WZH.bigWig","hct116-H3K27me3-ENCFF030SYQ.bigWig","hct116-H3K27ac-ENCFF225QAB.bigWig"]
#names = ["H3K9me3","H3K27me3","H3K27ac"]
dfs = []
mean_dfs = []
#lets make a pandas dataframe
for chrom in chrnames[chrstart:chrstop]:
    print(chrom)
    medi_dic = {}
    mean_dic = {}
    lastlength = 0
    for name,filen in zip(names,filenames):
        #print("Reading in: {0}".format(name))
        try:
            bw = pyBigWig.open("/Zulu/mike/chips/"+filen)
        except RuntimeError:
            print("Trouble opening {0}".format(filen))
        print(bw.chroms())
        prefix = commonprefix(bw.chroms().keys())
        #print(prefix)
        #calculate number of 25kb bins
        chromlength = bw.chroms(prefix+chrom)
        #print(chromlength)
        binnum = int(math.ceil(chromlength/small_resolution))
        print(binnum)
        smalls = bw.stats(prefix+chrom,nBins=binnum)
        medi_vals = []
        mean_vals = []
        for pos in range(resolution_ratio,binnum,resolution_ratio):
            chunk = np.array(smalls[pos-resolution_ratio:pos], dtype=np.float)
            try:
                medi_vals.append(np.median(chunk))
                mean_vals.append(np.mean(chunk))
            except TypeError:
                print(chunk)
        
        #print(chromlength)
        #print(binnum)
        #print(len(vals))
        #x = small_resolution
        #vals = []
        #while x < chromlength:
            #print(x)
            #vals.extend(bw.stats(prefix+chrom,x-small_resolution,x))
            #x += small_resolution
        #vals.extend(bw.stats(prefix+chrom,x-small_resolution,chromlength))
        #print(bw.stats("chr"+chrom,x-resolution,chromlength))
        #vals.extend([0,0,0,0,0])
        
        if lastlength:
            medi_vals.extend([0,0,0,0,0,0,0,0,0,0])
            medi_vals = medi_vals[:lastlength]
            mean_vals.extend([0,0,0,0,0,0,0,0,0,0])
            mean_vals = mean_vals[:lastlength]
        lastlength = len(medi_vals)
        #print(len(vals))
        #print(vals[:10])
        medi_dic[name] = medi_vals
        mean_dic[name] = mean_vals
        
    
    dfs.append(pd.DataFrame(data=medi_dic).fillna(0))
    mean_dfs.append(pd.DataFrame(data=mean_dic).fillna(0))
    
for df in dfs:
    print(df.shape)

['hct116-H3K9me3-ENCFF402WZH.bigWig', 'hct116-H3K27me3-ENCFF030SYQ.bigWig', 'hct116-H3K27ac-ENCFF225QAB.bigWig']
1
{'chrM': 16571, 'chr21': 48129895, 'chr5': 180915260, 'chr9': 141213431, 'chr8': 146364022, 'chr1': 249250621, 'chr4': 191154276, 'chr7': 159138663, 'chr22': 51304566, 'chr19': 59128983, 'chr6': 171115067, 'chr14': 107349540, 'chr17': 81195210, 'chrY': 59373566, 'chr13': 115169878, 'chr18': 78077248, 'chr3': 198022430, 'chr2': 243199373, 'chr11': 135006516, 'chr20': 63025520, 'chr15': 102531392, 'chr16': 90354753, 'chr12': 133851895, 'chr10': 135534747, 'chrX': 155270560}
2493


KeyboardInterrupt: 

In [ ]:
#ehre we need to double every signal on the X chromosome

if male == True:
    xindex = chrnames.index('X')
    print(dfs[xindex])
    dfs[xindex] = dfs[xindex] * 2
    print(dfs[xindex])

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (12,12)

hists = [[] for x in range(comp_num)]

print(comp_num)
count = 0
for df in dfs:
    count += 1
    plt.plot(df["H3K9me3"])
    plt.title(count)
    plt.show()
    plt.hist(df["H3K9me3"],bins=50)
    plt.title(count)
    plt.show()
    for x,column in enumerate(df[:comp_num]):
        #print(x)
        hists[x].extend(df[column].tolist())
    

for n,h in zip(names,hists):
    plt.hist(h,bins=50)
    plt.title(n)
    plt.show()
    
hists = [[] for x in range(comp_num)]    

for df in mean_dfs:
    #print(df)
    for x,column in enumerate(df[:comp_num]):
        #print(x)
        hists[x].extend(df[column].tolist())
    

for n,h in zip(names,hists):
    plt.hist(h,bins=50)
    plt.title(n)
    plt.show()




In [ ]:
matplotlib.rcParams['figure.figsize'] = (10,10)

#chrom = 1

start = 0
end = 1000

#plt.plot(dfs[chrom][0:1000])
#plt.plot(mean_dfs[chrom][0:1000][:comp_num])
#plt.ylim(0,10)
#plt.show()

In [ ]:
#here we will now implement a low-pass filter using fourier transform
"""
import numpy as np
import pandas as pd
import scipy

from scipy import signal



fft_fc = 0.1
fft_b = 0.08
fft_N = int(np.ceil((4 / fft_b)))
if not fft_N % 2: fft_N += 1
fft_n = np.arange(fft_N)


sinc_func = np.sinc(2 * fft_fc * (fft_n - (fft_N - 1) / 2.))
window = 0.42 - 0.5 * np.cos(2 * np.pi * fft_n / (fft_N - 1)) + 0.08 * np.cos(4 * np.pi * fft_n / (fft_N - 1))
sinc_func = sinc_func * window
sinc_func = sinc_func / np.sum(sinc_func)

s = list(df["Gro"])
fft_signal = np.convolve(s, sinc_func)
smooth_signal = list(df["Gro"].rolling(window=smooth,min_periods=1,center=True).mean())

xleft = 250
xright = 350

plt.plot(s[xleft:xright])
plt.plot(fft_signal[xleft:xright])
plt.plot(smooth_signal[xleft:xright])
#plt.legend()
plt.ylim(0,200)
plt.show()
"""

In [ ]:



#let's convert the fold changes over control to 'tiles. 'tiles aren't evenly spaced across range,
#instead each contains the same number of entries.

#this is quantiles, we want a more meaningful distribution though

quantiles = np.linspace(0,1,num=tiles,endpoint=True)
#print(quantiles)


signals = []

#first we just have to construct a big concatenated dataframe so we can call quantile on it.
#Then we just apply those quantiles to each individual chrom df.

concatdf = pd.concat(dfs)

#it would be nice if this worked but it doesn't so whatever.

print(concatdf.shape)
print((concatdf.sum(axis=1)))


#concatdf[(concatdf.sum(axis=1) != 0)]  
nozconcatdf = concatdf[concatdf.sum(axis=1) != 0]

#nozconcatdf = concatdf[(df.T != 0).any()]

print(nozconcatdf.shape)

wg_quantiles = nozconcatdf.quantile(quantiles)

print("quantiles")
print(wg_quantiles)

concatdf = []


for df in dfs:
    #df.replace(0, np.NaN,inplace=True)
    #for col in df:
        #col_smooth = col + '_smooth'
        #df[col_smooth] = df[col].rolling(window=smooth,min_periods=1,center=True).mean()
        #print(df[col])
        #print(df[col_smooth])
    
    #print(df)
    #print("q")
    #print(q)
    newcolumns = []
    for column in df.iloc[:,-comp_num:]:
        #print(df[column])
        #print(q[column])
        
        newcolumns.append([])
        for r in df[column]:
            place = wg_quantiles[column].searchsorted(r)#[0]
            newcolumns[-1].append(place)
        
    for index,column in enumerate(df.iloc[:,:comp_num]):
        df[column+"-tiles"] = pd.Series(newcolumns[index],dtype='int16')

#todo, more sophisticated tiling that uses a different dynamic range for each signal based on it's variance.

matplotlib.rcParams['figure.figsize'] = (5,5)

for col in dfs[0].iloc[:,-comp_num:]:
    df.hist(col,bins=range(tiles+1))
    plt.show()
    
for col in dfs[0].iloc[:,:comp_num]:
    df.hist(col,bins=200)
    plt.show()
    
#for col in dfs[1].iloc[:,-comp_num:]:
#    df.hist(col,bins=range(tiles+1))
#    plt.show()

#for x in range(quantiles):

#print(dfs)


#let's export those quantiles to a file
filename = "{0}-small:{1}-actual:{2}-{3}".format(celltype,small_resolution,resolution,"wgtiles")
wgt = open(filename,'w')

wg_quantiles.to_csv(wgt)


wgt.close()
print(filename)
! head -n 100 $filename


In [ ]:
def mapping(maps,sig1,sig2):
    total = 1
    for m,i,j in zip(maps,sig1,sig2):
        total += m[i,j]
    return total



def ncomp_mapping_update(matrix,sigs,maps,tiles,dists,distance_indices,slims,ignorebins,distance_min_bins):
    
    #lamb = 0.1
    
    #we need to iterate over all pixels in the matrix, but we just need to hold on to sums.
    import numpy as np
    cnum = len(sigs[0])
    mnum = len(matrix[0])
    gradmaps = []
    hessmaps = []
    countmaps = []
    newmaps = []
    for c in range(cnum):
        gradmaps.append(np.zeros((tiles,tiles)))
        hessmaps.append(np.zeros((tiles,tiles)))
        countmaps.append(np.zeros((tiles,tiles)))
        #newmaps.append(np.ones((tiles,tiles)))
        maps.append(np.array(maps[c]))
    for i in range(mnum):
        for j in range(i+distance_min_bins,mnum):
            if ignorebins[i] or ignorebins[j]:
                continue
            #we are iterating over the entire matrix
            distfactor = dists[distance_indices[j-i]]
            for c in range(cnum):
                #we are iterating over each signal or compartment
                si = sigs[i][c]
                sj = sigs[j][c]
                #mapped = mapping(maps[0:c]+maps[c+1:cnum],sigs[i][0:c]+sigs[i][c+1:cnum],sigs[j][0:c]+sigs[j][c+1:cnum])
                mapped = mapping(maps,sigs[i],sigs[j])
                
                o = np.nan_to_num(matrix[i][j])
                 
                grad = o/mapped - distfactor
                
                hess = -1 * o/(mapped*mapped)
                
                gradmaps[c][si,sj] += grad
                gradmaps[c][sj,si] += grad
                hessmaps[c][si,sj] += hess
                hessmaps[c][sj,si] += hess                
                
                
                #obsmaps[c][si,sj] += np.nan_to_num(matrix[i][j]/distfactor)
                #expmaps[c][si,sj] += mapped
                #obsmaps[c][sj,si] += np.nan_to_num(matrix[i][j]/distfactor)
                #expmaps[c][sj,si] += mapped
                #countmaps[c][si,sj] += 1
                #countmaps[c][sj,si] += 1
                #equat = (obsmaps[c][si,sj] - expmaps[c][si,sj])/countmaps[c][si,sj]
                #newmaps[c][si,sj] = (obsmaps[c][si,sj] - expmaps[c][si,sj])/countmaps[c][si,sj]
                #newmaps[c][sj,si] = (obsmaps[c][sj,si] - expmaps[c][sj,si])/countmaps[c][sj,si]
    
    for c in range(cnum):
        step = gradmaps[c]/hessmaps[c]
        #super_threshold_indices = hold < slims[0]
        #hold[super_threshold_indices] = slims[0]
        #add in a bit to deal with nans
        hold2 = maps[c] - step
        nan_indices = np.isnan(hold2)
        hold2[nan_indices] = slims[0]
        maps[c] = hold2
    
    #for c in range(cnum):
    #    newmaps[c] = (obsmaps[c] - expmaps[c])/countmaps[c]
        
    return maps,countmaps#,weirds


def ncomp_distance_update(d,matrix,sizebins,dists,sigs,maps,distance_ranges,ignorebins):
    #we need to iterate over all pixels in this range
    drange = distance_ranges[d]
    observed = 0
    expected = 0
    for i in range(sizebins):
        for j in range(i+drange[0],min(i+drange[1]+1,sizebins)):
            if ignorebins[i] or ignorebins[j]:
                continue
            observed += matrix[i,j]
            #this just happens to equate to the average since expected is 1 at beginning.
            expected += mapping(maps,sigs[i],sigs[j])
    #we now have observed and expected for every bin in range
    if not observed and not expected:
        #here we are out of range so just continue
        return 0
    return np.nan_to_num(observed/expected)


In [ ]:
def mu(X,B):
    v = np.ones(X.shape[0])
    #print(v.shape)
    #print(X.shape)
    #print(B.shape)
    #print((X @ B).shape)
    old = v + X @ B
    #print(old.shape)
    #print(H.shape)
    return old


def logL(Y,X,B):
    u = mu(X,B)
    return np.nansum((Y - u)**2)
    

def grad(Y,X,B):
    u = mu(X,B)
    #v = np.ones(X.shape[0])
    return -2 * X.T @ (Y - u)

def hess(Y,X,B):
    """
    from scipy.sparse import coo_matrix
    from scipy.sparse import csr_matrix
    u = mu(X,B)
    num = (Y)
    denom = (u*u)
    frac = (num / denom)
    #print('frac')
    #print(frac.shape)
    #refrac = frac[:, np.newaxis]
    #print(refrac.shape)
    #print(X.shape)
    diag_i = []
    diag_j = []
    diag_v = []
    for x,entry in enumerate(frac):
        diag_i.append(x)
        diag_j.append(x)
        diag_v.append(entry)

    coo_diag = coo_matrix((diag_v, (diag_i, diag_j)), shape=(Y.shape[0], Y.shape[0]))
    #print(coo_diag)
    csr_diag = csr_matrix(coo_diag)    
    one = (csr_diag @ X)
    """
    return (-2*X.T @ X)
    
    #return -1 * (((Y*H*H) / (u*u) @ X).T @ X)

In [ ]:
def MLE_ncomp(chrindex,resolution=resolution,distance_min=distance_min,comp_num=comp_num,tiles=tiles,
                     distance_min_bins=distance_min_bins,distance_max=distance_max,distance_max_bins=distance_max_bins,chrnames=chrnames,D=D,M=M,
                     clims=clims,slims=slims,blims=blims,correction=correction,
                     chrsizebins=chrsizebins,distance_ranges=distance_ranges,ignoreplots=ignoreplots,
                     distance_indices=distance_indices,tolerance=tolerance,ignore_threshold=ignore_threshold,
                     max_iter=max_iter,hic_directory=hic_directory,hic_prefix=hic_prefix,
                     ignorebins=ignorebins,dfs=dfs,chrstart=chrstart,chrstop=chrstop,
                     medianhighstd=medianhighstd,medianlowstd=medianlowstd,highstd=highstd,lowstd=lowstd,
                      nz_low_threshold=nz_low_threshold,nz_high_threshold=nz_high_threshold):
    #now we will iterate through each chromosome.
    #for each chromosome we load in the matrix
    #initialize all of the parameters
    #for every bin i + distance_min/resolution >= j
    #compute gradient for distance
    #then compute gradient for bias
    #then compute c-score
    #then compute error

    import numpy as np
    import pandas as pd
    import math
    import matplotlib.pyplot as plt
    from scipy.sparse import coo_matrix
    from scipy.sparse import csr_matrix
    import seaborn as sns
    from scipy import stats
    
    chrom1 = chrnames[chrindex]
    chrom2 = chrnames[chrindex]
    sizebins = chrsizebins[chrindex]
    print("Loading: {0} vs {1}".format(chrom1,chrom2))
    print("Size: {0}".format(sizebins))
    
    interactions = open(hic_directory+hic_prefix+chrom1+"_"+chrom2+"_"+str(resolution)+".txt",'r')
        
    #matrix = dok_matrix((sizebins,sizebins))
    matrix = np.zeros((sizebins,sizebins))
    print(matrix)
    
    for line in interactions:
        li = line.split()
        left = int(li[0])//resolution
        right = int(li[1])//resolution
        score = float(li[2])
        matrix[left,right] = np.nan_to_num(score)
        matrix[right][left] = np.nan_to_num(score)
        
        
    #plt.imshow(matrix,vmin = 0,vmax = 25)
    #plt.show()
    
    if "medians" in correction:
        medians = np.median(matrix,axis=0)
    
        median_mean = np.mean(medians[medians != 0])
        median_stddev = np.std(medians[medians != 0])
        
        print("Median Mean: {0}".format(median_mean))
        print("Median Std dev: {0}".format(median_stddev))
        high_line_data = np.array([median_mean+(median_stddev*medianhighstd) for i in range(len(medians))])
        low_line_data = np.array([median_mean-(median_stddev*medianlowstd) for i in range(len(medians))])
        if ignoreplots:
            plt.plot(medians)
            plt.plot(high_line_data, 'r--')
            plt.plot(low_line_data, 'r--')
            plt.show()
        medianignorebins = [(median_mean+(median_stddev*medianhighstd) < c) 
                      or median_mean-(median_stddev*medianlowstd) > c for c in medians]
    else:
        medianignorebins = [False for c in range(len(matrix[0]))]        

    
    
    if "sums" in correction:
        sums = np.sum(matrix,axis=0) + np.sum(matrix,axis=1)
        sum_mean = np.mean(sums[sums != 0])
        sum_stddev = np.std(sums[sums != 0])
        print("Sum Mean: {0}".format(sum_mean))
        print("Sum Std dev: {0}".format(sum_stddev))
        high_line_data = np.array([sum_mean+(sum_stddev*highstd) for i in range(len(sums))])
        low_line_data = np.array([sum_mean-(sum_stddev*lowstd) for i in range(len(sums))])
        sumymin = sum_mean+(sum_stddev*highstd*5)
        sumymax = sum_mean-(sum_stddev*lowstd*5)
        
        if ignoreplots:
            plt.plot(sums)
            plt.ylim(sumymin,sumymax)
            plt.plot(high_line_data, 'r--')
            plt.plot(low_line_data, 'r--')
            plt.show()
        sumignorebins = [(sum_mean+(sum_stddev*highstd) < c) or sum_mean-(sum_stddev*lowstd) > c for c in sums]
    else:
        sumignorebins = [False for c in range(len(matrix[0]))]
        
    
    if "nonzero" in correction:
        nonzerocounts = np.count_nonzero(matrix,axis=0)
        nonzero_mean = np.mean(nonzerocounts[nonzerocounts != 0])
        nonzero_stddev = np.std(nonzerocounts[nonzerocounts != 0])
        print("Nonzero Mean: {0}".format(nonzero_mean))
        print("Nonzero Std dev: {0}".format(nonzero_stddev))
        nonzeroignorebins = [(nonzero_mean+(nonzero_stddev*nz_high_threshold) < c) or
                      (nonzero_mean-(nonzero_stddev*nz_low_threshold) > c) for c in nonzerocounts]    
        if ignoreplots:
            plt.plot(nonzerocounts)
            high_line_data = np.array([nonzero_mean+(nonzero_stddev*nz_high_threshold) for i in range(len(nonzerocounts))])
            low_line_data = np.array([nonzero_mean-(nonzero_stddev*nz_low_threshold) for i in range(len(nonzerocounts))])   
            plt.plot(high_line_data, 'r--')
            plt.plot(low_line_data, 'r--')
            plt.show()        
    else:
        nonzeroignorebins = [False for c in range(len(matrix[0]))]  
        
    df = dfs[chrindex-chrstart]
    
    sigs = []
    
    #load in sigs so we can remove 0 quantiles
    signames = df.columns[-comp_num:].tolist()
    print(signames)
    for index, row in df.iloc[:,-comp_num:].iterrows():
        try:
            sigs.append([int(r) for r in row][-comp_num:])
        except ValueError:
            print("NaN")
            print(index)
            print(row)        
    print('sigs')
    #print(sigs)
    
    print(sigs[:10])
    #here we reverse sigs
    #signames.reverse()
    #holdsigs = [s.reverse() for s in sigs]
    #sigs = holdsigs
    #for s in sigs:
    #    s.reverse()
    #print(signames)
    #print(sigs[:10])
    
    
    if "quant" in correction:
        quantignorebins = [False for c in range(len(matrix[0]))]
        for n in range(len(signames)):
            print()
            for x in range(len(sigs)):
                if sigs[x][n] == 0:
                    quantignorebins[x] = True
    else:
        quantignorebins = [False for c in range(len(matrix[0]))]      
    
    ignorebins = [w | x | y | z for (w,x,y,z) in zip(medianignorebins, sumignorebins, nonzeroignorebins,quantignorebins)]
    print("Ignoring {0} median bins".format(sum(medianignorebins)))
    print("Ignoring {0} sum bins".format(sum(sumignorebins)))
    print("Ignoring {0} zero bins".format(sum(nonzeroignorebins)))
    print("Ignoring {0} quant bins".format(sum(quantignorebins)))

    #now we need to ignore z-norm outliers
    #first we set ignore rows and columns to nan
    #so they will be omitted by zscore operation
    nan_ignore_matrix = np.copy(matrix)
    nan_ignore_matrix[ignorebins,:] = np.nan
    nan_ignore_matrix[:,ignorebins] = np.nan
    
    #print(nan_ignore_matrix[0:10,0:10])
    
    
    zscoreignorebins = np.zeros(sizebins, dtype=bool)
    for d in range(sizebins):
        zscores = stats.zscore(np.diag(nan_ignore_matrix,d),nan_policy='omit')
        for e,z in enumerate(zscores):
            if z == np.nan:
                zscoreignorebins[e] = False
                zscoreignorebins[e+d] = False
            elif z > zscore_threshold:
                #make both row and column false
                zscoreignorebins[e] = True
                zscoreignorebins[e+d] = True
                
    #now we remove these bad bins
    print("Ignoring {0} zscore bins".format(sum(zscoreignorebins)))
    ignorebins = [x | y for (x,y) in zip(ignorebins,zscoreignorebins)]
    
    print("Ignoring {0} total bins".format(sum(ignorebins)))
    #now matrix is our entirely unnormalized matrix
    #initialize our variables:
    dists = list(D[:sizebins])
    maps = list(M[:])
    print("len of dists")
    print(len(dists))
    

    #for col in df.columns[:comp_num]:
    #    comps.append(list(df[col][:sizebins]))
    
    

    #for x in range(len(comps)):
    #    for y in range(len(comps[x])):
    #        if comps[x][y] < 0:
    #            comps[x][y] = 0
    #print(len(sigs[0]))
    #print(len(maps))
    
    #here we want to set True on ignore flag for any bins outside sig coverage
    #print("sigs")
    #print(len(sigs))
    #print("ignore")
    #print(len(ignorebins))

    if len(sigs) < len(ignorebins):
        for x in range(len(sigs),len(ignorebins)):
            #print(x)
            ignorebins[x] = True
    
    LLL = np.Inf
    count = 0
    error = 100000
    #d_error = 100
    #b_error = 100
    #c_error = 100
    
    #OK now let's define all our matrices:
    #to get y we need to compress to remove ignore bins, flatten,
    igno = np.where(ignorebins)[0]
    #print(igno)
    matrixdf = pd.DataFrame(matrix)
    #print("matrix")
    #print(matrixdf.shape)
    #matrixdf.drop(igno,axis=0,inplace=True)
    #matrixdf.drop(igno,axis=1,inplace=True)
    #plt.imshow(matrixdf,vmin=0,vmax=100)
    #plt.show()
    #print(ignorebins)
    #matrixdf.iloc[:,igno] = 0
    #matrixdf.iloc[igno,:] = 0    
    matrixdf.iloc[:,ignorebins] = np.nan
    matrixdf.iloc[ignorebins,:] = np.nan
    #plt.imshow(matrixdf,vmin=0,vmax=100)
    #plt.show()
    #now nan out the lower triangle:
    matrixdf = matrixdf.where(np.tril(np.triu(np.ones(matrixdf.shape),k=1+distance_min_bins),k=min(distance_max_bins,matrixdf.shape[0])).astype(np.bool))
    matrixdf = matrixdf.stack().reset_index()
    matrixdf.columns = ['i','j','v']
    #print(matrixdf)
    Y = matrixdf['v']
    #Y = np.reshape(np.array(Y),(Y.shape[0],1))
    
 
    #y = matrixdf.to_numpy.flatten()#observed values vector
    #B is now the learned coefficients. Basically the flattened maps. Constructed by comparing sigs to y
    #tiles * tiles * comp_num
    ttiles = tiles-1
    trisize = ((ttiles*(ttiles+1))//2)
    #print("sizes")
    #print(trisize)
    bsize = trisize*comp_num
    #print(bsize)
    B = np.zeros(bsize)
    #B[0] = 1
    #B = np.reshape(B,(bsize,1))
    
    #print('ysize')
    #print(Y.shape)
    #print('bsize')
    #print(bsize)    
    
    
    #lastly we need to define H. This is the expected value based on distance for every value.
    #we can initialize it as 0's and then learn it first.
    H = np.zeros(Y.shape[0])
    #H = np.reshape(H,(Y.shape[0],1))
    #to learn H you learn the full matrix, then you average every distance together
    H_dists = matrixdf['j'] - matrixdf['i']
    #print(H_dists)
    #print(H_dists)
    #print(H_dists[H_dists==1])
    Hadj = np.ones(len(dists))
    Hadj_exp = np.ones(Y.shape[0])
    
    #X is a CSR sparse matrix with each column corresponding to a sig-tile and each row a matrix bin.
    #X dimensions are y x B
    #once we have flattened y we can fill in each element in X. Should be 4 1's per row 
    #we construct by iterating over matrixdf, grabbing the appropriate sigs and adding that to a COO or DOK matrix.
    xijdic = {}
    for x in range(comp_num):
        for i in range(ttiles):
            for j in range(i,ttiles):
                y = trisize*x + np.sum(np.array(range(-1*i,0))+ttiles+1) + j-i
                xijdic[y] = [x,i,j]
    
    ii = []
    jj = []
    dd = []
    for index, row in matrixdf.iterrows():
        #first start with 1 in every row for B_0:
        #ii.append(index)
        #jj.append(0)
        #dd.append(1)
        #print(row)
        #print(row['i'])
        #print(row['j'])
        sigs_i = sigs[int(row['i'])]
        sigs_j = sigs[int(row['j'])]
        for x, (i,j) in enumerate(zip(sigs_i,sigs_j)):
            #here x is comp num, i is first quantile, j is second quantile
            #formula to get correct bin therefore is (1+tiles*tiles*x + tiles*i + j)
            if i > j:
                h = i
                i = j
                j = h
            ii.append(index)
            i = i-1
            j = j-1
            #jj.append(1+tiles*tiles*x + tiles*i + j)
            y = trisize*x + np.sum(np.array(range(-1*i,0))+ttiles+1) + j-i
            jj.append(y)
            dd.append(1)
        
        
    #print(xijdic)
    coo = coo_matrix((dd, (ii, jj)), shape=(Y.shape[0], bsize))
    #print(coo_matrix)
    X = csr_matrix(coo)
    #print('X')
    #print(np.sum(X,axis=0))
    #print(np.sum(X,axis=1))
    
    for d in range(1, len(dists)):
        #d_indices = np.where(H_dists == d)
        dmean = np.mean(Y[H_dists == d])
        H[H_dists == d] = dmean
        dists[d] = dmean

    #print(dists)
    #H = H1

    plt.plot(dists)
    plt.yscale('log')
    plt.xscale('log')
    plt.title("Distance Vector {0}".format(chrindex))
    plt.show()
    

    
    #let's compute the distance parameter
    #print(len(sigs))
    #print(len(ignorebins))
    #print(ignorebins[-10:])
    
    print("X {0}".format(X.shape))
    print("Y {0}".format(Y.shape))
    print("B {0}".format(B.shape))
    print("H {0}".format(H.shape))
    


    #plt.plot(dists)
    #plt.title("Distance Vector {0}".format(chrindex))
    #plt.show()
    #now we will begin iterating
    while error > tolerance and count < max_iter:
        #while count < max_iter:
        count += 1
        
        #olddists = np.copy(dists)
        #for d in range(len(dists)):
        #    dists[d] = ncomp_distance_update(d,matrix,sizebins,dists,sigs,maps,distance_ranges,ignorebins)
        #olddists = dists
        #plt.plot(olddists)
        #XB = (X @ B)
        #print(XB.shape)
        #print(XB)
        #H1 = Y/(X @ B)
        #print(H1)
        for d in range(1,len(dists)):
            #d_indices = H_dists[H_dists==d]
            XBmean = np.mean(mu(X,B)[H_dists==d])
            Hadj_exp[H_dists==d] = XBmean
            Hadj[d] = XBmean
        
        
        plt.plot(Hadj)
        plt.show()
        #now divide by H
        Yh = np.nan_to_num(Y/(H / Hadj_exp))
        #print(len(H))
        #print(len(Hadj_exp))
        #print(len(Y))
        #print(len(Yh))
        #print(Yh)
        print("len of dists")
        print(len(dists))
        plt.plot(dists)
        plt.plot(dists / Hadj)
        plt.yscale('log')
        plt.xscale('log')
        plt.show()
        
        #now we update B
        lamb = 0.1
        #G = grad(Y,X,B,H)
        #u  = H * (X @ B)
        #print("u")
        #print(u.shape)
        #G = X.T @ ((Y * H) / u) - X.T @ H
        Grad = grad(Yh,X,B)
        #He = hess(Y,X,B,H)
        #print(Grad.shape)
        #iGrad = np.linalg.inv(Grad)
        
        #u2 = (X @ B) * (X @ B)
        
        #yu2 = Y/u2
        
        #print(yu2.shape)
        #print(X.shape)
        
        #diag_i = []
        #diag_j = []
        #diag_v = []
        #for x,entry in enumerate(yu2.flatten()):
        #    diag_i.append(x)
        #    diag_j.append(x)
        #    diag_v.append(entry)
            
        #coo_diag = coo_matrix((diag_v, (diag_i, diag_j)), shape=(Y.shape[0], Y.shape[0]))
        #print(coo_diag)
        #csr_diag = csr_matrix(coo_diag)
        
        #print('csr')
        #print(csr_diag.shape)
        #print(yu2.shape)
        #print(X.shape)
        
        #xyu2 = csr_diag @ X
        #xyu2 = yu2.flatten() * X 
        #xyu2 = X @ yu2
        #print("xyu2")
        #print(xyu2.shape)
        
        #xxyu2 = xyu2.T @ X
        
        #print(xxyu2.shape)
        
        #Hess = -1 * (Y / ((X @ B) * (X @ B)) * X).T @ X
        #Hess = -1 * xxyu2.todense()
        Hess = hess(Yh,X,B).todense()
        #temp = (Y.flatten() / ((X.dot(B.flatten())) * (X.dot(B.flatten()))))[:, np.newaxis]
        #print('temp')
        #print(temp.shape)
        #print(X.shape)
        #Hess = -((temp * X).T).dot(X)
        
        #print(Hess.shape)
        #print(Hess)
        plt.imshow(Hess)
        plt.title('Hessian')
        plt.colorbar()
        plt.show()
        #plt.imshow(np.linalg.inv(Hess))
        #plt.title('inverse')
        #plt.colorbar()
        plt.show()
        plt.imshow(np.linalg.pinv(Hess))
        plt.title('pseudoinverse')
        plt.colorbar()
        plt.show()
        #B_new = B - (np.linalg.pinv(Hess) @ Grad)
        lamb = 1/max(np.absolute(Grad))
        #B_new = B + lamb*Grad
        inv = np.linalg.pinv(Hess)
        #print(inv.shape)
        step = -1 * (inv @ Grad[:, np.newaxis])
        #step = 1 * Grad * lamb
        #print(step.shape)
        #print(B.shape)
        B_new = B[:, np.newaxis] - step
        #B_new = B + step
        #print(B_new.shape)
        B = np.array(B_new).reshape(B.shape[0])
        #B = np.array(B_new)
        #print(Grad.shape)
        #print(B.shape)
        #here we should reshape B to visualize as 4 heatmaps.
        #inverse of this operation: 1+trisize*x + np.sum(np.array(range(-1*i,0))+tiles+1) + j
        G_reshape = np.zeros((comp_num,ttiles,ttiles))
        #print("G_0")
        #print(Grad[0])
        for n,g in enumerate(Grad):
            x,i,j = xijdic[n]
            G_reshape[x,i,j] = g
            G_reshape[x,j,i] = g
        
        #G_reshape = np.reshape(Grad[1:],(comp_num,tiles,tiles))
        #for g,n in zip(G_reshape,signames):
        #    sns.heatmap(g,center=0)
        #    plt.title("Gradient "+ n)
        #    plt.show()
            
        S_reshape = np.zeros((comp_num,ttiles,ttiles))
        #print("G_0")
        #print(Grad[0])
        for n,s in enumerate(step):
            x,i,j = xijdic[n]
            S_reshape[x,i,j] = s
            S_reshape[x,j,i] = s
        
        #G_reshape = np.reshape(Grad[1:],(comp_num,tiles,tiles))
        #print(len(S_reshape))
        #for s,n in zip(S_reshape,signames):
        #    sns.heatmap(s,center=0)
        #    plt.title("Step "+ n)
        #    plt.show()
            
        B_reshape = np.zeros((comp_num,ttiles,ttiles))
        #print("B_0")
        #print(B[0])
        for n,b in enumerate(B):
            x,i,j = xijdic[n]
            B_reshape[x,i,j] = b
            B_reshape[x,j,i] = b
        #B_reshape = np.reshape(B[1:],(comp_num,tiles,tiles))
        for b,n in zip(B_reshape,signames):
            sns.heatmap(b,center=0)
            plt.title("Maps "+ n)
            plt.show()
            
        
        
        #now update the maps
        #maps,counts = ncomp_mapping_update(matrix,sigs,maps,tiles,dists,distance_indices,slims,ignorebins,distance_min_bins)

        #print(maps)
        #print(counts)

        #for n,m,c in zip(signames,maps,counts):
        #    sns.heatmap(m,center=0)
        #    plt.title("Maps " + n)
        #    plt.show()
        #    #sns.heatmap(c)
        #    #plt.title("Counts " + n)
        #    #plt.show()

        #now let's compute log likelihood for graphing and testing purposes

        #LL = 0

        vis = np.zeros((sizebins,sizebins))
        visc = np.zeros((sizebins,sizebins))
        
        #LL = logL(Y,X,B,H)
        
        #u = mu(X,B,H)
        #logu = np.log(u)
        #print(np.any(np.isinf(u)))
        #print(np.any(np.isposinf(u)))
        #print(np.any(np.isneginf(u)))
        #LL = np.nansum(Y * logu - u)
        
        LL = logL(Yh,X,B)
        
        #plt.imshow(np.log(vis))
        #plt.show()     

        print("Log likelihood: {0}".format(LL))
        print("Last log likelihood: {0}".format(LLL))
        error = LLL-LL
        print("Chromosome {0} Iteration {1} Improvement: {2}".format(chrom1,count,error))
        LLL = LL
        
        
        expected = mu(X,B)
        observed = Yh
        expectedc = mu(X,B) * (H / Hadj_exp)
        observedc  = Y
        #print(observed)
        #print(expected)
        pearson = np.corrcoef(observed,expected)[0,1]
        print('Pearson: (0)'.format(pearson))
        
        for o,e,i,j in zip(observed,expected,matrixdf['i'],matrixdf['j']):
            vis[i,j] = e
            vis[j,i] = o
        for o,e,i,j in zip(observedc,expectedc,matrixdf['i'],matrixdf['j']):
            visc[i,j] = e
            visc[j,i] = o            
            
        matplotlib.rcParams['figure.figsize'] = (12,12)
        plt.imshow(vis)
        plt.colorbar()
        plt.show()
        
        plt.imshow(vis[100:200,100:200])
        plt.colorbar()
        plt.show()
        matplotlib.rcParams['figure.figsize'] = (6,6)

        matplotlib.rcParams['figure.figsize'] = (12,12)
        plt.imshow(visc,vmin=0,vmax=1000)
        plt.colorbar()
        plt.show()
        
        plt.imshow(visc[100:200,100:200],vmin=0,vmax=1000)
        plt.colorbar()
        plt.show()
        matplotlib.rcParams['figure.figsize'] = (6,6)
        #plt.plot(dists)
        #plt.title("Distance Vector {0}".format(chrindex))
        #plt.show()
        #plt.imshow(matrix)
        #plt.show()
        #plt.plot(bias)
        #plt.title("Bias Vector {0}".format(chrindex))
        #plt.show()
        #plt.plot(comps)
        #plt.title("C-Score {0}".format(chrindex))
        #plt.show()
        maps = B_reshape
        adj_dists = dists/Hadj
        print("len of dists")
        print(len(adj_dists))
    return (maps, sigs, adj_dists, ignorebins)


#chrindex = 10
#cs = MLE(chrindex,D,B,C,chrsizebins,distance_ranges,distance_indices,distance_min_bins,tolerance,ignore_threshold,max_iter)
#print(cs)

from ipyparallel import Client


matplotlib.rcParams['figure.figsize'] = (6,6)


#rc = Client()
#print(rc.ids)


#dv = rc[:]

#var_dict = ('chrnames'=chrnames,D,B,C,chrsizebins,distance_ranges,distance_indices,distance_min_bins,
#tolerance,ignore_threshold,max_iter),range(chrstart,chrstop))
#dv.push({'MLE_ncomp': MLE_ncomp,"ncomp_mapping_update":ncomp_mapping_update,
#        "mapping":mapping,"ncomp_distance_update":ncomp_distance_update})


#lv = rc.load_balanced_view()

msdi = []
for x in range(chrstart,chrstop):
    msdi.append(MLE_ncomp(x))

#msdi = dv.map_sync(lambda chrindex: MLE_ncomp(chrindex), range(chrstart,chrstop))
#print(cdb)


In [ ]:
#now we need to unpack c,d,b


mappings = []
signals = []
distances = []
ignored = []

for chromosome in msdi:
    mappings.append(chromosome[0])
    signals.append(chromosome[1])
    distances.append(chromosome[2])
    ignored.append(chromosome[3])
    


In [ ]:
dfnames = []
for c in dfs[0]:
    dfnames.append(c)

#temp
#we use names
    
#here let's export tile thresholds and mappings to file
for chrindex in range(chrstart,chrstop):
    corrindex = chrindex - chrstart
    #for x,signame in enumerate(dfnames[-comp_num:]):
    for x,signame in enumerate(dfnames[-comp_num:]):
        mapfileoutname = "/Zulu/mike/learnedmaps/{0}-{1}kb-chr{2}-{3}-learnedmaps-min:max-{4}kb:{5}kb.csv".format(celltype,resolution//1000,chrnames[chrindex],signame,distance_min//1000,distance_max//1000)
        #mapfileoutname = "/Zulu/mike/learnedmaps/{0}-{1}kb-chr{2}-{3}-tiles-learnedmaps-mindist-{4}kb.csv".format(celltype,resolution//1000,chrnames[chrindex],signame,distance_min//1000)

        mapdf = pd.DataFrame(mappings[corrindex][x])
        mapdf.to_csv(mapfileoutname,index=False)
        print(mapfileoutname)
        
    distancefileoutname = "/Zulu/mike/learneddistances/{0}-{1}kb-chr{2}-distances-min:max-{3}kb:{4}kb.csv".format(celltype,resolution//1000,chrnames[chrindex],distance_min//1000,distance_max//1000)
    distancedf = pd.DataFrame(distances[chrindex])
    distancedf.to_csv(distancefileoutname,index=False,na_rep=0)
    print(distancefileoutname)
    print(distancedf)
    ignorefileoutname = "/Zulu/mike/learnedignores/{0}-{1}kb-chr{2}-ignore.csv".format(celltype,resolution//1000,chrnames[chrindex])
    ignoredf = pd.DataFrame(ignored[corrindex])
    ignoredf.to_csv(ignorefileoutname,index=False)
    print(ignorefileoutname)
    #ignorefileoutname = "{0}-{1}kb-chr{2}-ignore.csv".format(celltype,resolution//1000,chrnames[chrindex])
    #ignoredf = pd.DataFrame(ignored[corrindex])
    #ignoredf.to_csv(ignorefileoutname,index=False)
    #print(ignorefileoutname)        
#! head $ignorefileoutname



In [ ]:
for x in range(chrstart,chrstop):
    plt.plot(distances[x])
plt.yscale('log')
plt.xscale('log')
plt.show()

In [ ]:
import seaborn as sns

matplotlib.rcParams['figure.figsize'] = (14,7)

#for maps in mappings:
#        for n,m in zip(dfs[0],maps):
#            sns.heatmap(m,center=0)
#            plt.title(n)
#            plt.show()

#for x in range(len(mappings)):


    
#print(dfnames[:comp_num])
for n,m1,m2 in zip(dfnames[-comp_num:],mappings[5],mappings[6]):
    fig, (ax1, ax2) = plt.subplots(1,2)
    #sns.heatmap((m1[:19,:19]), center=0, vmin =-1,vmax=1, ax=ax1)
    #sns.heatmap((m1[:19,:19]-(m1[:19,:19].mean()))/m1[:19,:19].std(ddof=1), center=0, vmin =-4,vmax=4, ax=ax1)
    #sns.heatmap((m1[:19,:19]-(m1[:19,:19].mean())), center=0, vmin =-1,vmax=1, ax=ax1)
    sns.heatmap((m1[:tiles,:tiles]), center=0, vmin =-1,vmax=1,ax=ax1)
    plt.title(n)
    #sns.heatmap((m2[:19,:19]), center=0, vmin =-1,vmax=1, ax=ax2)
    #sns.heatmap((m2[:19,:19]-(m2[:19,:19].mean()))/m2[:19,:19].std(ddof=1), center=0, vmin =-4,vmax=4, ax=ax2)
    #sns.heatmap((m2[:19,:19]-(m2[:19,:19].mean())), center=0, vmin =-1,vmax=1, ax=ax2)
    sns.heatmap((m2[:tiles,:tiles]), center=0, vmin =-1,vmax=1, ax=ax2)
    plt.title(n)
    plt.show()
    
#print(m1[:tiles,:tiles])